# Documentation

> Documentation of the functions used in building the OOD Metric Class

Visual overview of how the pipeline looks like. 

```{mermaid}
flowchart LR
  A[Feature\nEmbeddings] --> C{OOD Detection}
  B[In Distribution\nLabels] --> C
  C --> F[Uncertainty Score]
  F --> D[Out of Distribtuion]
  F --> E[In Distribtuion]
```

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from fastcore.utils import *
import numpy as np

## Utility Functions

In [ ]:
#| export
def compute_mean_and_covariance(
    embdedding: np.ndarray, # (n_sample, n_dim) n_sample - sample size of training set, n_dim - dimension of the embedding
    labels: np.ndarray, # (n_sample, ) n_sample - sample size of training set
) -> Tuple[np.ndarray, np.ndarray]: # Mean of dimension (n_dim, ) and Covariance matrix of dimension(n_dim, n_dim)
    """Computes class-specific means and a shared covariance matrix.
    """
    
    n_dim = embdedding.shape[1]
    class_ids = np.unique(labels)
    
    covariance = np.zeros((n_dim, n_dim)) 
    means = []

    def f(covariance, class_id):
        mask = np.expand_dims(labels == class_id, axis=-1) # to compute mean/variance use only those which belong to current class_id
        data = embdedding * mask
        mean = np.sum(data, axis=0) / np.sum(mask)
        diff = (data - mean) * mask
        covariance += np.matmul(diff.T, diff)
        return covariance, mean

    for class_id in class_ids:
        covariance, mean = f(covariance, class_id)
        means.append(mean)
        
    covariance = covariance / len(labels)
    return np.stack(means), covariance

In [ ]:
#| export
def compute_mahalanobis_distance(
    embdedding: np.ndarray, # Embdedding of dimension (n_sample, n_dim)
    means: np.ndarray, # A matrix of size (num_classes, n_dim), where the ith row corresponds to the mean of the fitted Gaussian distribution for the i-th class.
    covariance: np.ndarray # The shared covariance matrix of the size (n_dim, n_dim)
) -> np.ndarray: # A matrix of size (n_sample, n_class) where the (i, j) element corresponds to the Mahalanobis distance between i-th sample to the j-th class Gaussian.
    """Computes Mahalanobis distance between the input and the fitted Guassians. The Mahalanobis distance (Mahalanobis, 1936) is defined as

    $$distance(x, mu, sigma) = sqrt((x-\mu)^T \sigma^{-1} (x-\mu))$$

    where `x` is a vector, `mu` is the mean vector for a Gaussian, and `sigma` is
    the covariance matrix. We compute the distance for all examples in `embdedding`,
    and across all classes in `means`.

    Note that this function technically computes the squared Mahalanobis distance
    """
    
    covariance_inv = np.linalg.pinv(covariance)
    maha_distances = []

    def maha_dist(x, mean):
        # NOTE: This computes the squared Mahalanobis distance.
        diff = x - mean
        return np.einsum("i, ij, j->", diff, covariance_inv, diff)

    for x in embdedding:
        arr = []
        for mean in means:
            arr.append(maha_dist(x, mean))
        arr = np.stack(arr)
        maha_distances.append(arr)

    return np.stack(maha_distances)

## OOD Metric Computation Class

In [ ]:
#| export
class OODMetric:
    """OOD Metric Class that calculates the OOD scores for a batch of input embeddings.
    Initialises the class by fitting the class conditional gaussian using training data
    and the class independent gaussian using training data.
    """

    def __init__(self,
                 train_embdedding: np.ndarray, # An array of size (n_sample, n_dim) where n_sample is the sample size of training set, n_dim is the dimension of the embedding.
                 train_labels: np.ndarray # An array of size (n_train_sample, )
                ):
        self.means, self.covariance = compute_mean_and_covariance(train_embdedding, train_labels)
        self.means_bg, self.covariance_bg = compute_mean_and_covariance(train_embdedding, np.zeros_like(train_labels))

In [ ]:
#| export
@patch
def compute_rmd(
    self:OODMetric,
    embdedding: np.ndarray # An array of size (n_sample, n_dim), where n_sample is the sample size of the test set, and n_dim is the size of the embeddings.
) -> np.ndarray:  # An array of size (n_sample, ) where the ith element corresponds to the ood score of the ith data point.
    """This function computes the OOD score using the mahalanobis distance
    """
    
    distances = compute_mahalanobis_distance(embdedding, self.means, self.covariance)
    distances_bg = compute_mahalanobis_distance(embdedding, self.means_bg, self.covariance_bg)

    rmaha_distances = np.min(distances, axis=-1) - distances_bg[:, 0]
    return rmaha_distances

## Example

In [ ]:
train_embedding = np.random.standard_normal((32, 2048))
train_labels = np.random.randint(low=0, high=5, size=(32,))

In [ ]:
test_eq(type(train_embedding), np.ndarray) # check that embeddings are a numpy array
test_eq(type(train_labels), np.ndarray) # check that labels are numpy array
test_eq(train_labels.dtype, int) # check that labels are integers only
test_eq(train_labels.ndim, 1) # check that labels is one dimensional
test_eq(train_embedding.shape[0], train_labels.shape[0]) # check n_samples are same

In [ ]:
ood = OODMetric(train_embedding, train_labels)

In [ ]:
test_eq(ood.means.shape[0], len(np.unique(train_labels))) # for each unique class, we should get one mean embedding
test_eq(ood.means.shape[1], train_embedding.shape[1]) # size of mean vector should be the same of the size of embedding

test_eq(ood.covariance.shape[0], train_embedding.shape[1]) # covariance matrix should be of size n_dim, n_dim
test_eq(ood.covariance.shape[1], train_embedding.shape[1])

test_eq(ood.means_bg.shape[0], 1)
test_eq(ood.means_bg.shape[1], train_embedding.shape[1])

test_eq(ood.covariance_bg.shape[0], train_embedding.shape[1])
test_eq(ood.covariance_bg.shape[1], train_embedding.shape[1])

### Outputs

In [ ]:
ood.compute_rmd(train_embedding) # testing on the train embedding itself

array([-7.12871042e+11, -7.12871042e+11, -4.79065270e+11, -4.79065270e+11,
       -3.67675472e+12, -3.67675472e+12, -3.67675472e+12, -3.67675472e+12,
       -3.67675472e+12, -3.67675472e+12, -4.79065270e+11, -7.12871042e+11,
       -3.67675472e+12, -7.12871042e+11, -7.12871042e+11, -3.67675472e+12,
       -3.67675472e+12, -4.79065270e+11, -7.12871042e+11, -3.67675472e+12,
       -4.79065270e+11, -7.12871042e+11, -3.67675472e+12, -3.67675472e+12,
       -7.12871042e+11, -4.79065270e+11, -3.10000000e+01, -3.67675472e+12,
       -7.12871042e+11, -3.67675472e+12, -3.67675472e+12, -7.12871042e+11])

In [ ]:
test_embedding = np.random.standard_normal((32, 2048)) # test embedding from same distribution
scores = ood.compute_rmd(test_embedding)
print(scores)

[-2.05771726e+13 -2.27167064e+12 -3.15972809e+13 -2.83203671e+13
 -8.02122950e+12  5.93743984e+12 -3.67828101e+11 -3.16653199e+13
 -1.36345972e+13  3.30371127e+12 -4.53200326e+12 -2.30555589e+13
 -7.09979234e+12 -7.09262508e+12 -3.14463722e+13 -1.02980640e+13
 -2.54963601e+13 -3.70189789e+13 -3.58977889e+13 -1.38412972e+12
 -2.00452276e+13  1.06730332e+13 -1.59431950e+13  1.93239518e+12
 -1.04444115e+11 -4.73320512e+13 -3.56125723e+13 -8.80546424e+12
 -2.96015093e+13  3.31277321e+11 -6.27571552e+12 -1.87232415e+13]


In [ ]:
test_eq(scores.shape[0], test_embedding.shape[0])
test_eq(scores.ndim, 1)

In [ ]:
test_embedding = np.random.uniform(size=(32, 2048)) # test embedding from different distribution
ood.compute_rmd(test_embedding)

array([-3.54006118e+12, -8.65916412e+12, -4.73688435e+12, -1.60718635e+12,
       -1.72445557e+13, -3.36838664e+12, -6.60664867e+12, -9.13608411e+12,
       -1.23727464e+13, -5.85111616e+12, -1.31499281e+12,  1.90948607e+12,
       -3.02551028e+12, -1.46485214e+13, -1.22621818e+13, -2.71230431e+12,
       -2.14953115e+12, -1.49131895e+13, -7.76931721e+12, -5.08826447e+12,
       -9.76403621e+12, -1.05555622e+13, -1.05103160e+13, -6.48915780e+12,
       -5.34682692e+12, -1.40335367e+13, -4.06784378e+12, -5.02260109e+12,
       -6.08178220e+12, -5.52765883e+12,  1.59961404e+12, -1.56397685e+13])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()